In [820]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

Описание задачи
Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок "Смотрели ранее" - в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корреляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке - например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).

Входные данные

Вам дается две выборки с пользовательскими сессиями - id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ; после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

1) считали данные

In [847]:
train = pd.read_csv('coursera_sessions_train.txt', sep=';', header=None)
train.columns = ['Viewed', 'Bought']

test = pd.read_csv('coursera_sessions_test.txt', sep=';', header=None)
test.columns = ['Viewed', 'Bought']

# Train

In [848]:
train.head()

,Viewed,Bought
0,"0,1,2,3,4,5",NaN
1,"9,10,11,9,11,12,9,11",NaN
2,"16,17,18,19,20,21",NaN
3,"24,25,26,27,24",NaN
4,"34,35,36,34,37,35,36,37,38,39,38,39",NaN


Сессии, в которых пользователь ничего не купил, исключаем из оценки качества. Если товар не встречался в обучающей выборке, его популярность равна 0. Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров. Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k / precision@k.



Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)

2) посчитали частоты встреч

In [849]:
viewed_frequences = pd.Series(','.join(train.Viewed.values).split(',')).value_counts()
bought_frequences = pd.Series(','.join(train.Bought.dropna().values).split(',')).value_counts()

3) товары, которые не куплены - куплены 0 раз

In [850]:
bought_frequences = bought_frequences[viewed_frequences.index].fillna(0)

In [851]:
train.dropna(inplace=True)

Если частота одинаковая, то сортировать нужно по возрастанию момента просмотра (чем раньше появился в просмотренных, тем меньше приоритет)

In [852]:
Viewed_sort = []
Bought_sort = []
#for i in train.Viewed.index:
#for i in [7]:
for i in train.Viewed.index:
    temp_list = np.unique(train.Viewed[i].split(','))[::-1]
    #print(viewed_frequences[temp_list])
    Viewed_sort.append(sorted(temp_list, key=lambda x: viewed_frequences[x], reverse =True))
    Bought_sort.append(sorted(temp_list, key=lambda x: bought_frequences[x], reverse =True))

In [853]:
train['Viewed_sort'] = Viewed_sort
train['Bought_sort'] = Bought_sort

In [854]:
train.head()

,Viewed,Bought,Viewed_sort,Bought_sort
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","[63, 64, 68, 67, 66, 65, 61, 60, 62, 59]","[67, 63, 60, 68, 66, 65, 64, 62, 61, 59]"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"[85, 93, 89, 90, 92, 84, 86, 87, 91, 88]","[86, 93, 85, 92, 91, 90, 89, 88, 87, 84]"
19,"138,198,199,127",199,"[127, 138, 198, 199]","[199, 138, 127, 198]"
30,"303,304,305,306,307,308,309,310,311,312",303,"[303, 306, 310, 309, 307, 304, 312, 311, 308, ...","[303, 312, 311, 310, 309, 308, 307, 306, 305, ..."
33,"352,353,352",352,"[352, 353]","[352, 353]"


In [855]:
def count_precision(temp_bought, recommend, n = 1):
    count = 0
    for i in range(len(recommend)):
        for j in range(len(temp_bought)):
            #print(recommend[i], temp_bought[j])
            if recommend[i] == temp_bought[j]:
                #print(recommend[i], temp_bought[j])
                count += 1
    #print(temp_bought, recommend,count/len(recommend))
    return(count/n)

In [856]:
def count_recall(temp_bought, recommend):
    count = 0
    for i in range(len(recommend)):
        for j in range(len(temp_bought)):
            if recommend[i] == temp_bought[j]:
                count += 1
    return(count/len(temp_bought))

In [857]:
train.index

Int64Index([    7,    10,    19,    30,    33,    55,    64,    72,    89,
               93,
            ...
            49883, 49890, 49896, 49908, 49932, 49943, 49964, 49981, 49991,
            49995],
           dtype='int64', length=3608)

In [858]:
viewed_presision_5 = []
viewed_recall_5 = []
viewed_presision_1 = []
viewed_recall_1 = []
bought_presision_5 = []
bought_recall_5 = []
bought_presision_1 = []
bought_recall_1 = []

for i in train.index:
    viewed_presision_5.append(count_precision(train.Bought[i].split(','), train.Viewed_sort[i][:5], 5))
    viewed_presision_1.append(count_precision(train.Bought[i].split(','), train.Viewed_sort[i][:1]))
    viewed_recall_5.append(count_recall(train.Bought[i].split(','), train.Viewed_sort[i][:5]))
    viewed_recall_1.append(count_recall(train.Bought[i].split(','), train.Viewed_sort[i][:1]))

    bought_presision_5.append(count_precision(train.Bought[i].split(','), train.Bought_sort[i][:5], 5))
    bought_presision_1.append(count_precision(train.Bought[i].split(','), train.Bought_sort[i][:1]))
    bought_recall_5.append(count_recall(train.Bought[i].split(','), train.Bought_sort[i][:5]))
    bought_recall_1.append(count_recall(train.Bought[i].split(','), train.Bought_sort[i][:1]))

In [859]:
def write_to_file(List, name_file):
    file = open(name_file,'w')
    file.write(str(round(List[0],2))+ ' '+ str(round(List[1],2)) + ' ' + str(round(List[2],2)) +' ' + str(round(List[3],2)))
    file.close()

In [860]:
List = []
print('recall_1, precision_1, recall_5, precision_5')
List.append(np.mean(np.array(viewed_recall_1)))
List.append(np.mean(np.array(viewed_presision_1)))
List.append(np.mean(np.array(viewed_recall_5)))
List.append(np.mean(np.array(viewed_presision_5)))
write_to_file(List, 'train_viewed.txt')
print('train_viewed')
print(np.round(np.array(List),2))

List = []
List.append(np.mean(np.array(bought_recall_1)))
List.append(np.mean(np.array(bought_presision_1)))
List.append(np.mean(np.array(bought_recall_5)))
List.append(np.mean(np.array(bought_presision_5)))
write_to_file(List, 'train_bought.txt')
#print(List)
print('train_bought')
print(np.round(np.array(List),2))

recall_1, precision_1, recall_5, precision_5
train_viewed
[0.44 0.51 0.82 0.21]
train_bought
[0.68 0.79 0.93 0.25]


# Test

In [862]:
test.dropna(inplace=True)

In [863]:
viewed_frequences_test = pd.Series(','.join(test.Viewed.values).split(',')).value_counts()

In [864]:
new_viewed_frequences_test = viewed_frequences[viewed_frequences_test.index].fillna(0)
new_bought_frequences_test  = bought_frequences[viewed_frequences_test.index].fillna(0)

In [865]:
Viewed_sort_test = []
Bought_sort_test = []
#for i in train.Viewed.index:
#for i in [7]:
for i in test.Viewed.index:
    temp_list = np.unique(test.Viewed[i].split(','))[::-1]
    #print(viewed_frequences[temp_list])
    Viewed_sort_test.append(sorted(temp_list, key=lambda x: new_viewed_frequences_test[x], reverse =True))
    Bought_sort_test.append(sorted(temp_list, key=lambda x: new_bought_frequences_test[x], reverse =True))

In [866]:
test['Viewed_sort'] = Viewed_sort_test
test['Bought_sort'] = Bought_sort_test

In [867]:
test.head()

,Viewed,Bought,Viewed_sort,Bought_sort
7,"63,68,69,70,66,61,59,61,66,68","66,63","[63, 68, 66, 61, 59, 70, 69]","[63, 70, 69, 68, 66, 61, 59]"
14,"158,159,160,159,161,162",162,"[158, 162, 160, 159, 161]","[158, 162, 160, 161, 159]"
19,"200,201,202,203,204","201,205","[204, 202, 203, 200, 201]","[204, 202, 203, 201, 200]"
34,"371,372,371","371,373","[371, 372]","[372, 371]"
40,422,422,[422],[422]


In [868]:
viewed_presision_5 = []
viewed_recall_5 = []
viewed_presision_1 = []
viewed_recall_1 = []
bought_presision_5 = []
bought_recall_5 = []
bought_presision_1 = []
bought_recall_1 = []

for i in test.index:
    viewed_presision_5.append(count_precision(test.Bought[i].split(','), test.Viewed_sort[i][:5], 5))
    viewed_presision_1.append(count_precision(test.Bought[i].split(','), test.Viewed_sort[i][:1]))
    viewed_recall_5.append(count_recall(test.Bought[i].split(','), test.Viewed_sort[i][:5]))
    viewed_recall_1.append(count_recall(test.Bought[i].split(','), test.Viewed_sort[i][:1]))

    bought_presision_5.append(count_precision(test.Bought[i].split(','), test.Bought_sort[i][:5], 5))
    bought_presision_1.append(count_precision(test.Bought[i].split(','), test.Bought_sort[i][:1]))
    bought_recall_5.append(count_recall(test.Bought[i].split(','), test.Bought_sort[i][:5]))
    bought_recall_1.append(count_recall(test.Bought[i].split(','), test.Bought_sort[i][:1]))

In [869]:
List = []
print('recall_1, precision_1, recall_5, precision_5')
List.append(np.mean(np.array(viewed_recall_1)))
List.append(np.mean(np.array(viewed_presision_1)))
List.append(np.mean(np.array(viewed_recall_5)))
List.append(np.mean(np.array(viewed_presision_5)))
write_to_file(List, 'test_viewed.txt')
print('test_viewed')
print(np.round(np.array(List),2))

List = []
List.append(np.mean(np.array(bought_recall_1)))
List.append(np.mean(np.array(bought_presision_1)))
List.append(np.mean(np.array(bought_recall_5)))
List.append(np.mean(np.array(bought_presision_5)))
write_to_file(List, 'test_bought.txt')
#print(List)
print('test_bought')
print(np.round(np.array(List),2))

recall_1, precision_1, recall_5, precision_5
test_viewed
[0.41 0.48 0.8  0.2 ]
test_bought
[0.41 0.47 0.79 0.2 ]
